In [ ]:
# Standard Libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle 
from sklearn.model_selection import train_test_split
import os
%matplotlib inline

# Neural Network Libraries
import torch
import torch.nn as nn
import torchvision
from torch.autograd import Variable
from torchvision.utils import make_grid
import torch.nn.functional as F
from torchvision import datasets, transforms
import scipy.linalg as linalg

#for colab purposes only
from google.colab import drive
drive.mount('/content/gdrive')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
batch_size = 32

transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.5], [0.5]) 
])

train_dataset = datasets.MNIST('./data', 
                               train=True, 
                               download=True, 
                               transform=transform)

validation_dataset = datasets.MNIST('./data', 
                                    train=False, 
                                    transform=transform)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

validation_loader = torch.utils.data.DataLoader(dataset=validation_dataset, 
                                                batch_size=batch_size, 
                                                shuffle=False)


In [ ]:
for (X_train, y_train) in train_loader:
    print('X_train:', X_train.size(), 'type:', X_train.type())
    print('y_train:', y_train.size(), 'type:', y_train.type())
    break

X_train: torch.Size([32, 1, 28, 28]) type: torch.FloatTensor
y_train: torch.Size([32]) type: torch.LongTensor


In [ ]:
# Courtesy https://github.com/CSCfi/machine-learning-scripts.git

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 50)
        self.fc1_drop = nn.Dropout(0.2)
        self.fc2 = nn.Linear(50, 50)
        self.fc2_drop = nn.Dropout(0.2)
        self.fc3 = nn.Linear(50, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = self.fc1_drop(x)
        x = F.relu(self.fc2(x))
        x = self.fc2_drop(x)
        return F.log_softmax(self.fc3(x), dim=1)

model = Net().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
criterion = nn.CrossEntropyLoss()

In [ ]:
def train(epoch, log_interval=200):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad() 
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

In [ ]:
def validate(loss_vector, accuracy_vector):
    model.eval()
    val_loss, correct = 0, 0
    for data, target in validation_loader:
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        val_loss += criterion(output, target).data.item()
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    val_loss /= len(validation_loader)
    loss_vector.append(val_loss)

    accuracy = 100. * correct.to(torch.float32) / len(validation_loader.dataset)
    accuracy_vector.append(accuracy)
    
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        val_loss, correct, len(validation_loader.dataset), accuracy))

In [ ]:
%%time
epochs = 10

lossv, accv = [], []
for epoch in range(1, epochs + 1):
    train(epoch)
    validate(lossv, accv)


Validation set: Average loss: 0.2976, Accuracy: 9121/10000 (91%)


Validation set: Average loss: 0.2293, Accuracy: 9304/10000 (93%)


Validation set: Average loss: 0.1959, Accuracy: 9389/10000 (94%)


Validation set: Average loss: 0.1643, Accuracy: 9470/10000 (95%)


Validation set: Average loss: 0.1547, Accuracy: 9509/10000 (95%)


Validation set: Average loss: 0.1400, Accuracy: 9558/10000 (96%)


Validation set: Average loss: 0.1342, Accuracy: 9578/10000 (96%)


Validation set: Average loss: 0.1282, Accuracy: 9604/10000 (96%)


Validation set: Average loss: 0.1307, Accuracy: 9594/10000 (96%)


Validation set: Average loss: 0.1273, Accuracy: 9597/10000 (96%)

CPU times: user 4min 13s, sys: 701 ms, total: 4min 14s
Wall time: 4min 18s


In [ ]:
modelPath = '/content/gdrive/MyDrive/Colab Notebooks/GAN'
modelfile = open(os.path.join(modelPath,'mnistModel.pkl'), 'wb')
pickle.dump(model, modelfile)
modelfile.close()